<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Isomorphic Strings

In [ ]:
import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

public class isomorphicString {
    public static void main(String[] args) {
        String s = "paper";
        String t = "title";

        if (s.length() != t.length()) {
            System.out.println("No");
        }

        Map<Character, Character> mapSToT = new HashMap<>();
        Map<Character, Character> mapTToS = new HashMap<>();

        for (int i = 0; i < s.length(); i++) {
            char charS = s.charAt(i);
            char charT = t.charAt(i);
            if (mapSToT.containsKey(charS)) {
                if (mapSToT.get(charS) != charT) {
                    System.out.println("No");
                }
            } else {
                mapSToT.put(charS, charT);
            }
            if (mapTToS.containsKey(charT)) {
                if (mapTToS.get(charT) != charS) {
                    System.out.println("No");
                }
            } else {
                mapTToS.put(charT, charS);
            }
        }
        System.out.println("Yes");
    }
}


ValidAnagram

In [ ]:
import java.util.HashMap;

public class validAnagram {
    public static void main(String[] args) {
        String s ="anagaram";
        String t ="naagaram";
        if(s == null || t == null || s.length()!=t.length()){
            System.out.println("false");
        }
        HashMap<Character,Integer> sd = new HashMap<>();
        HashMap<Character,Integer> td = new HashMap<>();
        for(int i=0; i<s.length(); i++){
            char charS = s.charAt(i);
            char charT = t.charAt(i);
            sd.put(charS, sd.getOrDefault(charS, 0) + 1);
            td.put(charT, td.getOrDefault(charT, 0) + 1);
        }
        for (char key : sd.keySet()) {
            if (!td.containsKey(key) || !sd.get(key).equals(td.get(key))) {
                System.out.println("false");
            }
        }
        System.out.println("true");
    }
}
